### Set the auth toke and site

In [1]:
const token = 'My auth token';
const site = 'da-dc';

async function logError(response) {
  console.error(`Request failed with status: ${response.status} ${response.statusText}`);
  
  console.error('Response Headers:');
  for (const [key, value] of response.headers.entries()) {
    console.error(`  ${key}: ${value}`);
  }
  
  try {
    const errorData = await response.json();
    console.error('Error Response Body:', errorData);
  } catch (e) {
    const errorText = await response.text();
    console.error('Error Response Body (text):', errorText);
  }
  
  throw new Error(`HTTP ${response.status}: ${response.statusText}`);
}

### Get the sitemap config from EDS

In [ ]:
const response = await fetch(`https://admin.hlx.page/config/adobecom/sites/${site}/content/sitemap.yaml`, {
  headers: {
    'x-auth-token': token,
  }
});

if (!response.ok) {
  logError(response);
}

const outputFile = `${site}_sitemap.yaml`;
const sitemapCfg = await response.text();
await Deno.writeTextFile(outputFile, sitemapCfg);
console.log(sitemapCfg);

### Update the sitemap config

In [ ]:
const inputFile = `${site}_sitemap.yaml`;
const sitemapCfg = await Deno.readTextFile(inputFile);

const response = await fetch(`https://admin.hlx.page/config/adobecom/sites/${site}/content/sitemap.yaml`, {
  method: 'POST',
  headers: {
    'x-auth-token': token,
    'content-type': 'text/yaml'
  },
  body: sitemapCfg,
});

if (response.ok) {
  console.log(response.status);
  const sitemapCfgUpdated = await response.text();
  console.log(sitemapCfgUpdated);
} else {
  logError(response);
}

### Force update of sitemap.xml

* If sitemap.xml files are generated successfully, it returns a lists of sitemap.xml paths.
* If failed, check the error message.

In [ ]:
const response = await fetch(`https://admin.hlx.page/sitemap/adobecom/${site}/main/dc-shared/assets/sitemap.xml`, {
  method: 'POST',
  headers: {
    'x-auth-token': token,
  }
});

if (!response.ok) {
  logError(response);
} else {
  const sitemap = await response.text();
  console.log(sitemap);
}